In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolorsx
from matplotlib.patches import Circle

import os
from glob import glob

from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.utils import proj_plane_pixel_scales
from astropy.wcs.utils import pixel_to_skycoord, skycoord_to_pixel
from astropy.nddata import Cutout2D

from reproject import reproject_exact, reproject_interp

from AstroColour.AstroColour import RGB

import matplotlib.patheffects as patheffects
import matplotlib.patheffects as PathEffects

%matplotlib widget

In [ ]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
fig_width_pt = 244.0  # Get this from LaTeX using \the\columnwidth
text_width_pt = 508.0 # Get this from LaTeX using \the\textwidth

inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt*1.5 # width in inches
fig_width_full = text_width_pt*inches_per_pt*1.5  # 17
fig_height =fig_width*golden_mean # height in inches
fig_size = [fig_width,fig_height] #(9,5.5) #(9, 4.5)
fig_height_full = fig_width_full*golden_mean

In [ ]:
path = '/Users/zgl12/Python_Scripts/Image_Processing/MAST/MAST_2025-11-25T20_43_45.354Z/'

name = f'jw06882-o038_t041_nircam_clear-f115w_i2d.fits'

hdu = fits.open(path+name)
data = hdu[1].data
header = hdu[1].header
hdr = hdu[0].header
wcs = WCS(header)
# hdu.info()
# print(repr(header))
hdu.close()

colour = np.load('/Users/zgl12/sneos_rgb.npy')

In [ ]:
# plt.figure()
# plt.imshow(colour, origin='lower')
# plt.show()

In [ ]:
# colour.shape, data.shape

In [ ]:
wcs.pixel_scale_matrix

In [ ]:
def compute_output_shape(w_old, w_new, ny, nx):
    # Corners of original image
    corners = np.array([
        [0,     0    ],
        [nx-1,  0    ],
        [0,     ny-1 ],
        [nx-1,  ny-1]
    ])

    # Convert to sky, then into new WCS pixel coords
    sky = pixel_to_skycoord(corners[:,0], corners[:,1], w_old)
    x_new, y_new = skycoord_to_pixel(sky, w_new)

    # compute bounding box
    xmin, xmax = np.min(x_new), np.max(x_new)
    ymin, ymax = np.min(y_new), np.max(y_new)

    # Return required integer shape
    new_nx = int(np.ceil(xmax - xmin))
    new_ny = int(np.ceil(ymax - ymin))

    # Also return pixel shift so CRPIX is correctly placed
    shift_x = -xmin
    shift_y = -ymin

    return new_ny, new_nx, shift_x, shift_y

w = wcs.celestial
ny, nx = data.shape

# --- build full valid north-up WCS FIRST ---
new_wcs = WCS(naxis=2)
new_wcs.wcs.crval = w.wcs.crval
new_wcs.wcs.crpix = [nx/2, ny/2]
new_wcs.wcs.ctype = ["RA---TAN", "DEC--TAN"]

scale = 8.67346751159964e-06  # arcsec/pixel in degrees
new_wcs.wcs.cdelt = [scale, -scale]   # East-left, North-up

# identity PC matrix
new_wcs.wcs.pc = np.eye(2)

# --- NOW compute new output shape ---
new_ny, new_nx, sx, sy = compute_output_shape(w, new_wcs, ny, nx)

# shift CRPIX to keep image in frame
new_wcs.wcs.crpix += [sx, sy]

# --- reproject color cube ---
new_colour = np.zeros((new_ny, new_nx, 3), dtype=colour.dtype)

for i in range(3):
    new_colour[:,:,i], _ = reproject_interp((colour[:,:,i], w), new_wcs,
                                            shape_out=(new_ny, new_nx))

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

fig = plt.figure(figsize=(fig_width_full/1.2, fig_width_full/1.2))
axs = fig.add_subplot(1,1,1, projection=new_wcs)
axs.imshow(new_colour)   # usual orientation
axs.invert_xaxis()
axs.invert_yaxis()

center = SkyCoord("19h31m49.5s", "-26d34m30s")   # add your Dec here if known

# --- 2. Offsets ---
dra = 25 * u.arcsec        # RA half-width
ddec = 25 * u.arcsec       # Dec half-width

# RA offsets must be applied as angles on the sphere
west  = SkyCoord(center.ra  - dra, center.dec)   # smaller RA → west
east  = SkyCoord(center.ra  + dra, center.dec)   # larger RA → east
south = SkyCoord(center.ra, center.dec - ddec)
north = SkyCoord(center.ra, center.dec + ddec)

# --- 3. Convert to pixel coords ---
cx_w, cy_s  = new_wcs.world_to_pixel(west)
cx_e, cy_n  = new_wcs.world_to_pixel(east)
cx_c, cy_c  = new_wcs.world_to_pixel(center)
sx, sy      = new_wcs.world_to_pixel(south)
nx_, ny_    = new_wcs.world_to_pixel(north)

# --- 4. Apply limits ---
# RA axis is x; Dec axis is y.
# Note: x axis may be inverted if East is left.

axs.set_xlim(cx_e, cx_w)   # left = East (larger RA), right = West (smaller RA)
axs.set_ylim(sy, ny_)      # y increases upward (Dec increasing)


# Arrow lengths
dRA = (10 * u.arcsec)  # East arrow length
dDec = (10 * u.arcsec) # North arrow length

# Compute end points
end_points = SkyCoord(center.ra - dra, center.dec - ddec, frame='icrs')

arrow_east  = SkyCoord(center.ra + dRA - dra, center.dec - ddec, frame='icrs')
arrow_north = SkyCoord(center.ra - dra, center.dec + dDec - ddec, frame='icrs')

# Convert back to pixel coords
cnx, cny     = new_wcs.world_to_pixel(end_points)
ex, ey     = new_wcs.world_to_pixel(arrow_east)
nx_, ny_   = new_wcs.world_to_pixel(arrow_north)

# Draw arrows
axs.annotate('', xy=(ex+40, ey-40), xytext=(cnx+40, cny-40),
            arrowprops=dict(arrowstyle='->', lw=2, color='white'))
axs.text(ex+70, ey - 32, r'\textbf{E}', color='white')

axs.annotate('', xy=(nx_+40, ny_-40), xytext=(cnx+40, cny-40),
            arrowprops=dict(arrowstyle='->', lw=2, color='white'))
axs.text(nx_ + 55, ny_ - 45, r'\textbf{N}', color='white', fontsize=12)

axs.text(ex+6, ey - 70, r"$10.0''.=46.6\;{\rm kpc}$", color='white')

axs.text(ex+200, ey - 1550, r"JWST/NIRCam -- Sep 2025", color='white')

start_x = ex + 420
start_y = ey - 1520

pieces = [
    (r"F115W$+$F150W", "cornflowerblue"),
    (" / ", "white"),
    (r"F200W$+$F277W", "limegreen"),
    (" / ", "white"),
    (r"F356W$+$F444W", "red"),
]

fig.canvas.draw()
renderer = fig.canvas.get_renderer()

for txt, color in pieces:
    
    if txt == " / ":
        start_x += 19
    t = axs.text(
        start_x, start_y,
        txt,
        color=color,
        va="top", ha="left",
        transform=axs.transData,
        clip_on=False,
        zorder=1000,
    )

    t.set_path_effects([
        PathEffects.withStroke(linewidth=1, foreground="black", alpha=0.3),
        PathEffects.Normal()
    ])

    fig.canvas.draw()
    disp_bbox = t.get_window_extent(renderer=renderer)
    data_bbox = axs.transData.inverted().transform_bbox(disp_bbox)

    # Advance to the next piece but FIX font right-side bearing
    start_x += data_bbox.width

plt.axis('off')
plt.show()

In [ ]:
# data_to_save = np.random.random((100, 100))
# hdu = fits.PrimaryHDU(data_to_save, header = None)
# hdul = fits.HDUList([hdu])
# hdul[0].header['FILTER'] = 'White'
# hdul.writeto('new_file.fits', overwrite=True)
# hdul.close()